In [1]:
import pandas as pd
import numpy as np
import openai
import os
import requests
import json


%load_ext autoreload
%autoreload 2

## Set API Keys

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

# Test LangChain's Chatbot
Test Langchain's chatbot system

#### Setup our Chatbot object
We'll use davinci-003 for now since it's cheaper for testing. Certain tuning functionality might not be available for this 3.5-turbo yet.

In [3]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# model_version = "text-davinci-003"
model_version = "gpt-3.5-turbo"

# llm = OpenAI(temperature=0, model_name = model_version)
llm = ChatOpenAI(temperature=0, model_name = model_version)

### Load our data

In [4]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
loader = UnstructuredPDFLoader("../documents/wftda_rules.pdf")

In [6]:
data = loader.load()

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [7]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 190743 characters in your document


### Chunk things into smaller documents
This will make it easier to use embeddings, making the model more efficient.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
print (f'Now you have {len(texts)} documents')

Now you have 219 documents


### Setup Embeddings
This will let us use vectorized representations of our documents, which will make it easier to find similar documents. It will ultimately reduce the amount of informatoin we feed into the model, which can help reduce API costs.

In [9]:
# The vectorstore we'll be using
from langchain.vectorstores import FAISS

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA

# The easy document loader for text
from langchain.document_loaders import TextLoader

# The embedding engine that will convert our text to vectors
from langchain.embeddings.openai import OpenAIEmbeddings

In [10]:
# Get your embeddings engine ready
embeddings = OpenAIEmbeddings()

# Embed your documents and combine with the raw text in a pseudo db. Note: This will make an API call to OpenAI
docsearch = FAISS.from_documents(texts, embeddings)

In [11]:
# Create the retrieval engine
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                 retriever=docsearch.as_retriever())

## Time to query Autoref

In [ ]:
query = "We have someone who wants to be a Jam Referee. Can you give me a list of \
    some of the more important things to keep in mind when jam refereeing?"
response = qa.run(query)
print(response)

In [15]:
print(response)

Sure! Here are some important things to keep in mind when jam refereeing:

1. Communicate clearly with the skaters and other officials.
2. Make sure the jammers are properly identified and communicate to them that they are the jammer for that jam.
3. Ensure that the jammers are wearing the star on their helmet and starting in the correct position.
4. Watch for illegal actions by the jammers, such as cutting the track or initiating contact outside of the engagement zone.
5. Signal who is the lead jammer and how many points each jammer earns on each trip through the pack.
6. Keep track of the time and call off the jam when necessary.
7. Be aware of the skaters' positions on the track and watch for penalties.
8. Stay focused and alert throughout the entire jam.


# Make the Chat Bot
Single queries are nice, but we want to be able to hold a conversation with it. This will make it easier to get assistance for more complex situations that can arise in Roller Derby where you might need a back and forth.

In [16]:
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate

# Chat specific components
from langchain.memory import ConversationBufferMemory

In [17]:
template = f"""
You are an expert referee for the sport Roller Derby. Your goal is to help the user understand
the rules of Roller Derby, as well as assist in adjuticating any disputes confusing situations
that might occur where the rules are unclear or tricky to apply. You are friendly and helpful,
but also kind of sassy, because this is Roller Derby after all.
Please give references to the rulebook when possible.

{chat_history}
Human: {human_input}
Chatbot:"""
"""